# Business Case Example using Tensorflow 1.14

# 1) Preprocess the data

##  a) Extract the data from csv

In [4]:
import numpy as np 
from sklearn import preprocessing

In [6]:
raw_csv_data = np.loadtxt('Audiobooks-data.csv', delimiter=',')

#excluding the first and the last column
unscaled_inputs_all = raw_csv_data[:,1:-1]

#last column
targets_all = raw_csv_data[:,-1]

## b) Balance the dataset

In [7]:
#Calculating the number of targets that are 1's
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

#deleting all extra 0's
unscaled_input_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

## c) Standarize the input

In [10]:
scaled_inputs = preprocessing.scale(unscaled_input_equal_priors)

## d) Shuffle the data

In [11]:
#shuffling the indices
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

In [12]:
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## e) Split the dataset into train, validation and test

In [14]:
#80-10-10
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

In [19]:
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count :]
test_targets = shuffled_targets[train_samples_count+validation_samples_count :]

In [23]:
# Checking if we have balanced the data

print(np.sum(train_targets),  train_samples_count,  np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets),  validation_samples_count,  np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),  test_samples_count,  np.sum(test_targets)/test_samples_count)

1769.0 3579 0.49427214305671974
221.0 447 0.49440715883668906
247.0 448 0.5513392857142857


## f) Save the three datset in .*npz

In [24]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)